In [ ]:
# install quantum espresso from conda
# !conda install conda-forge::qe
# !pip install ase

In [ ]:
from ase.io import read
from ase.calculators.espresso import Espresso, EspressoProfile

# Load CIF
struct = read("Mn0.25Zn0.75(FeO2)2.cif")

# Pseudopotentials keyed by real symbols (ASE reuses the same UPF for Fe, Fe1, etc.)
pseudos = {
    "Fe": "Fe.pbe-spn-kjpaw_psl.1.0.0.UPF",
    "Mn": "Mn.pbe-spn-kjpaw_psl.0.3.1.UPF",
    "Zn": "Zn.pbe-dnl-kjpaw_psl.1.0.0.UPF",
    "O": "O.pbe-n-kjpaw_psl.1.0.0.UPF",
}

profile = EspressoProfile(command="/opt/conda/bin/pw.x", pseudo_dir="./pseudos")

# Stabilized SCF: smearing + local-TF mixing + higher ecutrho
input_data = {
    "control": {
        "calculation": "scf",
        "etot_conv_thr": 5.6000000000e-03,
        "forc_conv_thr": 1.0000000000e-03,
        "outdir": "./out/",
        "prefix": "aiida",
        "pseudo_dir": "./pseudos/",
        "tprnfor": True,
        "tstress": True,
        "verbosity": "high",
    },
    "system": {
        "degauss": 2.7500000000e-02,
        "ecutrho": 1.0800000000e+03,
        "ecutwfc": 9.0000000000e+01,
        "ibrav": 0,
        "nat": 56,
        "nosym": False,
        "nspin": 2,
        "ntyp": 4,
        "occupations": "smearing",
        "smearing": "cold",
        "starting_magnetization(1)": 3.1250000000e-01,
        "starting_magnetization(2)": 3.3333333333e-01,
        "starting_magnetization(3)": 1.0000000000e-01,
        "starting_magnetization(4)": 1.0000000000e-01
    },
    "electrons": {
        "conv_thr": 2.2400000000e-08,
        "electron_maxstep": 80,
        "mixing_beta": 4.0000000000e-01,
    },
}

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudos,
    kpts=None,  # Γ-only, QE uses Γ-optimized path
    input_data=input_data,
)

struct.calc = calc
E_ev = struct.get_potential_energy()
print("SCF energy (eV):", E_ev)

### Update calculator for Band calculation

In [ ]:
fermi_level = calc.get_fermi_level()

In [ ]:
# Get the lattice and band path
lattice = struct.get_cell()
bandpath = lattice.bandpath(npoints=50)
input_data['control'].update({'calculation':'bands','restart_mode':'restart','verbosity':'high'})
# Set up the calculator for band structure calculation

In [ ]:
struct.calc = Espresso(profile=profile,
    pseudopotentials=pseudo_dict, kpts={'path': bandpath.kpts},
    input_data=input_data)


### Start Band calculation

In [ ]:
struct.calc.calculate(struct, [], [])

In [ ]:
from ase.spectrum.band_structure import BandStructure, get_band_structure
bands = get_band_structure(struct, reference=fermi_level)

In [ ]:
bands.plot(show=True, emin=fermi_level - 5, emax=fermi_level + 5.0)

In [ ]:
! ls